# **IMPORTING LIBRARIES**

In [45]:
import numpy as np
import pandas as pd 
import string
import matplotlib.pyplot as plt
import csv
import joblib
import pandas as pd
import numpy as np
import xgboost as xgb
from tqdm import tqdm
from sklearn.svm import SVC
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping
from nltk import word_tokenize
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import accuracy_score, confusion_matrix
stop_words = stopwords.words('english')

# **DATA SANITY**

In [2]:
#Loading Data
dataset = pd.read_csv(r'C:\Users\ayush.bajpayee\Desktop\Amazon Hackathon\dataset\train.csv', escapechar='\\',
                      quoting = csv.QUOTE_NONE)

In [4]:
#Do not Tamper this df_original
df_original = dataset.copy()

In [32]:
# #DO NOT RUN
# dataset_to_be_saved = dataset.copy()
# dataset_to_be_saved.to_csv('ModifiedTRAIN.csv', index = False)

In [35]:
dataset.head()

,TITLE,DESCRIPTION,BULLET_POINTS,BRAND,BROWSE_NODE_ID
0,"Pete The Cat Bedtime Blues Doll, 14.5 Inch","Pete the Cat is the coolest, most popular cat ...","[Pete the Cat Bedtime Blues plush doll,Based o...",MerryMakers,0
1,"The New Yorker NYHM014 Refrigerator Magnet, 2 ...",The New Yorker Handsome Cello Wrapped Hard Mag...,[Cat In A Tea Cup by New Yorker cover artist G...,The New Yorker,1
2,The Ultimate Self-Sufficiency Handbook: A Comp...,NaN,Skyhorse Publishing,imusti,2
3,Amway Nutrilite Kids Chewable Iron Tablets (100),NaN,"[Nutrilite Kids,Chewable Iron Tablets,Quantity...",Amway,3
4,Teacher Planner Company A4 6 Lesson Academic T...,NaN,NaN,NaN,4


In [38]:
dataset.shape

(2903024, 5)

In [12]:
dataset.columns

Index(['TITLE', 'DESCRIPTION', 'BULLET_POINTS', 'BRAND', 'BROWSE_NODE_ID'], dtype='object')

In [11]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2903024 entries, 0 to 2903023
Data columns (total 5 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   TITLE           object
 1   DESCRIPTION     object
 2   BULLET_POINTS   object
 3   BRAND           object
 4   BROWSE_NODE_ID  int64 
dtypes: int64(1), object(4)
memory usage: 110.7+ MB


In [27]:
dataset.isna().mean()

TITLE             0.000024
DESCRIPTION       0.249279
BULLET_POINTS     0.057272
BRAND             0.019544
BROWSE_NODE_ID    0.000000
dtype: float64

In [25]:
dataset.isna().sum()

TITLE                 71
DESCRIPTION       723664
BULLET_POINTS     166263
BRAND              56737
BROWSE_NODE_ID         0
dtype: int64

In [12]:
print(dataset['TITLE'][0])
print('\n')
print(dataset['TITLE'][1])
print('\n')
print(dataset['TITLE'][2])
print('\n')
print(dataset['TITLE'][3])
print('\n')
print(dataset['TITLE'][4])

Pete The Cat Bedtime Blues Doll, 14.5 Inch


The New Yorker NYHM014 Refrigerator Magnet, 2 x 3.5


The Ultimate Self-Sufficiency Handbook: A Complete Guide to Baking, Crafts, Gardening, Preserving Your Harvest, Raising Animals and More


Amway Nutrilite Kids Chewable Iron Tablets (100)


Teacher Planner Company A4 6 Lesson Academic Teacher Planner - Leatherette Pink


In [11]:
print(dataset['BULLET_POINTS'][0])
print('\n')
print(dataset['BULLET_POINTS'][1])
print('\n')
print(dataset['BULLET_POINTS'][2])
print('\n')
print(dataset['BULLET_POINTS'][3])
print('\n')
print(dataset['BULLET_POINTS'][4])
print('\n')
print(dataset['BULLET_POINTS'][5])

[Pete the Cat Bedtime Blues plush doll,Based on the popular Pete the Cat books by James Dean,Super cuddly and ready for naptime and bedtime,Safe for all ages, perfect for ages 2 and up,Measures 14. 5 inches]


[Cat In A Tea Cup by New Yorker cover artist Gurbuz Dogan Eksioglu,Handsome cello wrapped hard magnet,Ideal for home or office, or for a gift and for any New Yorker magazine lover,It highlight one of the many beautiful New Yorker covers in full color,Rigid magnet measures 2. 5-Inch width by 3. 5-Inch height]


Skyhorse Publishing


[Nutrilite Kids,Chewable Iron Tablets,Quantity 100 tablets]


nan


[Color: Blue,Sleeve: Full Sleeve,Material: Cotton,Neck: Round]


In [22]:
dataset['BULLET_POINTS'].apply(type).value_counts()

<class 'str'>      2736761
<class 'float'>     166263
Name: BULLET_POINTS, dtype: int64

In [17]:
dataset_onlyfloat = dataset.copy()

In [31]:
dataset_onlyfloat['BULLET_POINTS'].fillna(value = 'OTHERS', inplace=True)

In [32]:
dataset_onlyfloat.isna().sum()

TITLE                 71
DESCRIPTION       723664
BULLET_POINTS          0
BRAND              56737
BROWSE_NODE_ID         0
dtype: int64

In [29]:
dataset_onlyfloat[dataset_onlyfloat['BULLET_POINTS'].notna()]

,TITLE,DESCRIPTION,BULLET_POINTS,BRAND,BROWSE_NODE_ID
0,"Pete The Cat Bedtime Blues Doll, 14.5 Inch","Pete the Cat is the coolest, most popular cat ...","[Pete the Cat Bedtime Blues plush doll,Based o...",MerryMakers,0
1,"The New Yorker NYHM014 Refrigerator Magnet, 2 ...",The New Yorker Handsome Cello Wrapped Hard Mag...,[Cat In A Tea Cup by New Yorker cover artist G...,The New Yorker,1
2,The Ultimate Self-Sufficiency Handbook: A Comp...,NaN,Skyhorse Publishing,imusti,2
3,Amway Nutrilite Kids Chewable Iron Tablets (100),NaN,"[Nutrilite Kids,Chewable Iron Tablets,Quantity...",Amway,3
5,Men'S Full Sleeve Raglan T-Shirts Denim T-Shir...,Men'S Full Sleeve Raglan T-Shirts Denim T-Shir...,"[Color: Blue,Sleeve: Full Sleeve,Material: Cot...",Bhavya Enterprise,5
...,...,...,...,...,...
2903018,The NXT LVL Liquid Silicone Soft Back Cover Ca...,Product color may slightly vary due to photogr...,[Liquid silicone case compatible with apple pr...,Generic,794481
2903019,Premium Aviator Sunglasses - HD Polarized (Bri...,These premium Aviator Sunglasses with 5 color ...,"[Frame size: Lens height - 56mm, Lens width - ...",Generic,1040
2903020,Social Distance Stickers - Set of 5 Sticker Sl...,set of 5 prints social distancing sticker self...,[covid19 safety sticker - set of 5 to maintain...,Generic,15199
2903021,Torr-to Face Shield PACK OF 5 with Adjustable ...,* COMPLETE FACE PROTECTION: Torr-to Face Shiel...,"[350 MICRONS PACK OF 5 PCS,COMPLETE FACE PROTE...",TORR-TO,1044933


In [23]:
dataset_onlyfloat[dataset_onlyfloat['BULLET_POINTS'].apply(type) == type('csdv')]

,TITLE,DESCRIPTION,BULLET_POINTS,BRAND,BROWSE_NODE_ID
0,"Pete The Cat Bedtime Blues Doll, 14.5 Inch","Pete the Cat is the coolest, most popular cat ...","[Pete the Cat Bedtime Blues plush doll,Based o...",MerryMakers,0
1,"The New Yorker NYHM014 Refrigerator Magnet, 2 ...",The New Yorker Handsome Cello Wrapped Hard Mag...,[Cat In A Tea Cup by New Yorker cover artist G...,The New Yorker,1
2,The Ultimate Self-Sufficiency Handbook: A Comp...,NaN,Skyhorse Publishing,imusti,2
3,Amway Nutrilite Kids Chewable Iron Tablets (100),NaN,"[Nutrilite Kids,Chewable Iron Tablets,Quantity...",Amway,3
5,Men'S Full Sleeve Raglan T-Shirts Denim T-Shir...,Men'S Full Sleeve Raglan T-Shirts Denim T-Shir...,"[Color: Blue,Sleeve: Full Sleeve,Material: Cot...",Bhavya Enterprise,5
...,...,...,...,...,...
2903018,The NXT LVL Liquid Silicone Soft Back Cover Ca...,Product color may slightly vary due to photogr...,[Liquid silicone case compatible with apple pr...,Generic,794481
2903019,Premium Aviator Sunglasses - HD Polarized (Bri...,These premium Aviator Sunglasses with 5 color ...,"[Frame size: Lens height - 56mm, Lens width - ...",Generic,1040
2903020,Social Distance Stickers - Set of 5 Sticker Sl...,set of 5 prints social distancing sticker self...,[covid19 safety sticker - set of 5 to maintain...,Generic,15199
2903021,Torr-to Face Shield PACK OF 5 with Adjustable ...,* COMPLETE FACE PROTECTION: Torr-to Face Shiel...,"[350 MICRONS PACK OF 5 PCS,COMPLETE FACE PROTE...",TORR-TO,1044933


In [34]:
dataset['BRAND'].apply(type).value_counts()

<class 'str'>      2846287
<class 'float'>      56737
Name: BRAND, dtype: int64

# **DATA PREPROCESSING**

## Dropping Description

In [20]:
dataset.drop(columns='DESCRIPTION', inplace = True)

## Handling Duplicates

In [5]:
#Handling Duplicates
dataset.drop_duplicates(inplace=True)

## Handling **Nulls**

All nulls in all columns, will be replaced by **'OTHERS'**

In [7]:
#Filling Nulls with OTHERS
dataset.fillna('OTHERS', inplace=True)

In [8]:
dataset.isna().sum()

TITLE             0
DESCRIPTION       0
BULLET_POINTS     0
BRAND             0
BROWSE_NODE_ID    0
dtype: int64

## Handling **BULLET_POINTS**

BULLET_POINTS contains a string either **containing** '[]' denoting it is a list of strings or **not containing** '[]' denoting there is only one Bullet Point. Will convert Bullet Points into That specific format

In [9]:
print(dataset['BULLET_POINTS'][0])
print('\n')
print(dataset['BULLET_POINTS'][1])
print('\n')
print(dataset['BULLET_POINTS'][2])
print('\n')
print(dataset['BULLET_POINTS'][3])
print('\n')
print(dataset['BULLET_POINTS'][4])
print('\n')
print(dataset['BULLET_POINTS'][5])

[Pete the Cat Bedtime Blues plush doll,Based on the popular Pete the Cat books by James Dean,Super cuddly and ready for naptime and bedtime,Safe for all ages, perfect for ages 2 and up,Measures 14. 5 inches]


[Cat In A Tea Cup by New Yorker cover artist Gurbuz Dogan Eksioglu,Handsome cello wrapped hard magnet,Ideal for home or office, or for a gift and for any New Yorker magazine lover,It highlight one of the many beautiful New Yorker covers in full color,Rigid magnet measures 2. 5-Inch width by 3. 5-Inch height]


Skyhorse Publishing


[Nutrilite Kids,Chewable Iron Tablets,Quantity 100 tablets]


OTHERS


[Color: Blue,Sleeve: Full Sleeve,Material: Cotton,Neck: Round]


In [10]:
#Replacing ] and [ with ''
dataset['BULLET_POINTS'] = dataset['BULLET_POINTS'].str.replace('[', '')
dataset['BULLET_POINTS'] = dataset['BULLET_POINTS'].str.replace(']', '')

<ipython-input-10-88901d48649c>:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  dataset['BULLET_POINTS'] = dataset['BULLET_POINTS'].str.replace('[', '')
<ipython-input-10-88901d48649c>:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  dataset['BULLET_POINTS'] = dataset['BULLET_POINTS'].str.replace(']', '')


In [11]:
print(dataset['BULLET_POINTS'][0])
print('\n')
print(dataset['BULLET_POINTS'][1])
print('\n')
print(dataset['BULLET_POINTS'][2])
print('\n')
print(dataset['BULLET_POINTS'][3])
print('\n')
print(dataset['BULLET_POINTS'][4])
print('\n')
print(dataset['BULLET_POINTS'][5])

Pete the Cat Bedtime Blues plush doll,Based on the popular Pete the Cat books by James Dean,Super cuddly and ready for naptime and bedtime,Safe for all ages, perfect for ages 2 and up,Measures 14. 5 inches


Cat In A Tea Cup by New Yorker cover artist Gurbuz Dogan Eksioglu,Handsome cello wrapped hard magnet,Ideal for home or office, or for a gift and for any New Yorker magazine lover,It highlight one of the many beautiful New Yorker covers in full color,Rigid magnet measures 2. 5-Inch width by 3. 5-Inch height


Skyhorse Publishing


Nutrilite Kids,Chewable Iron Tablets,Quantity 100 tablets


OTHERS


Color: Blue,Sleeve: Full Sleeve,Material: Cotton,Neck: Round


In [12]:
#Converting BULLET_POINTS from comma seperated values to list containing values
dataset['BULLET_POINTS'] = dataset['BULLET_POINTS'].map(lambda x: x.split(','))

In [13]:
dataset['BULLET_POINTS'].apply(type).value_counts()

<class 'list'>    2801530
Name: BULLET_POINTS, dtype: int64

## Dumping/Loading

In [14]:
#Dumping
joblib.dump('V1', compress='zlib')

In [33]:
#Loading
dataset = joblib.load('V1')

In [37]:
dataset_original = dataset.copy()
dataset_temp = dataset.iloc[:10000,:].copy()

# **NLP**

## Preprocessing

In [35]:
# Tokenize
dataset_temp['TITLE'] = dataset_temp['TITLE'].apply(word_tokenize)

In [29]:
def remove_punc(x):
    x = [''.join(c for c in s if c not in string.punctuation) for s in x]
    x = [s for s in x if s]
    return x

dataset_temp['TITLE'] = dataset_temp['TITLE'].apply(remove_punc)

C:\Users\AYUSH~1.BAJ\AppData\Local\Temp/ipykernel_4940/4247752062.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_temp['TITLE'] = dataset_temp['TITLE'].apply(remove_punc)


In [38]:
dataset_temp

,TITLE,BULLET_POINTS,BRAND,BROWSE_NODE_ID
0,"Pete The Cat Bedtime Blues Doll, 14.5 Inch","[Pete the Cat Bedtime Blues plush doll, Based ...",MerryMakers,0
1,"The New Yorker NYHM014 Refrigerator Magnet, 2 ...",[Cat In A Tea Cup by New Yorker cover artist G...,The New Yorker,1
2,The Ultimate Self-Sufficiency Handbook: A Comp...,[Skyhorse Publishing],imusti,2
3,Amway Nutrilite Kids Chewable Iron Tablets (100),"[Nutrilite Kids, Chewable Iron Tablets, Quanti...",Amway,3
4,Teacher Planner Company A4 6 Lesson Academic T...,[OTHERS],OTHERS,4
...,...,...,...,...
9998,Brass Tabla Set Brass Material … (Hammered Bra...,[Brass Bayan (Bass - Left Metal Drum) Head 9 t...,Fasherati,2603
9999,Max Men's Slim Fit Jeans (WI18CDGV08_Mid Blue_28),"[Color Name: Mid Blue, Material: Cotton, Zip f...",Max,40
10000,Gems Gallery Natural & Pure Stone Crystal Bead...,[Rare and Precious gemstone like rose quartz a...,Gems Gallery,1010
10001,Max Men's Checkered Slim Fit Formal Shirt (FBW...,"[Color Name: White, Material: Cotton, Long sle...",Max,1254


In [39]:
X = dataset_temp['TITLE']
y = dataset_temp['BROWSE_NODE_ID']

In [41]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42)

In [43]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer

nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])
nb.fit(X_train, y_train)

Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', MultinomialNB())])

In [47]:
%%time
from sklearn.metrics import classification_report
y_pred = nb.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
# print(classification_report(y_test, y_pred))

accuracy 0.138
Wall time: 548 ms


In [50]:
dataset_temp = dataset.iloc[:100000,:].copy()
dataset_temp.shape

(100000, 4)

In [51]:
X = dataset_temp['TITLE']
y = dataset_temp['BROWSE_NODE_ID']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42)

from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer

nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])
nb.fit(X_train, y_train)

%%time
from sklearn.metrics import classification_report
y_pred = nb.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
# print(classification_report(y_test, y_pred))

MemoryError: Unable to allocate 2.63 GiB for an array with shape (353640000,) and data type float64